In [1]:
# Dylan Kriegman / May 2023
# modified by Andrea Miller / Fall 2023

# usb port for dynamixel: tty.usbmodem14101

# idea: failutre recovery << screws example


In [2]:
import pyrealsense2 as rs
import open3d as o3d
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
import numpy as np
import math
import cv2 as cv
import scipy.ndimage as nd
# Used to display Matplotlib plots in Jupyter
# %matplotlib inline
import matplotlib.pyplot as plt
import time

# PIL used to save images as pngs"
from PIL import Image

# Robotics Toolbox used to determine camera coordinate frame given joint angles
import roboticstoolbox as rtb

# Spatial Math is used for manipulating geometric primitives
import spatialmath as sm
from spatialmath import SE3

# Poses is from rmlib and used for converting between 4 x 4 homogenous pose and 6 element vector representation (x,y,z,rx,ry,rz)
import poses
 
import copy

import swift

from spatialgeometry.geom.CollisionShape import (CollisionShape,Mesh,Cylinder,Cuboid,Box,Sphere)

import spatialgeometry as sg

# UR Interface
import rtde_control
import rtde_receive

# Gripper Interface 
from Motor_Code import Motors

# For insterfacing with fast downward solver
import os

# This class was created for this project. Connects to the gripper and has some test functions. - Jensen
#import UR5_Interface as ur

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import serial.tools.list_ports
ports = []
for port in serial.tools.list_ports.comports():
    ports.append(port.name)
print(ports)

['ttyS4', 'ttyS0', 'ttyACM0']


In [4]:
# Call this once to intialize serial connections to ur and gripper
# To list serial ports of the motor interface
# $ python -m serial.tools.list_ports


robotIP = "192.168.0.6"
con = rtde_control.RTDEControlInterface(robotIP)
rec = rtde_receive.RTDEReceiveInterface(robotIP)

# original: servoPort = "/dev/ttyACM0"
# use 'ls /dev' in terminal to find port address being used by dynamixel motor
servoPort = "/dev/ttyACM0"

# original: gripperController = Motors(servoPort)
gripperController = Motors(servoPort)

gripperController.torquelimit(600)


Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 


In [5]:
# con.disconnect()
# rec.disconnect()

In [6]:
import UR5_Interface as ur
try:
    ur = ur.UR5_Interface() #-- commneted out since I changed the class to an import -Jensen
    ur.c,ur.r,ur.gripperController = con,rec,gripperController
    ur.testRoutine()
    
    
except Exception as e:
    raise(e)
finally:
    pass
    # ur.c.disconnect()
    # ur.r.disconnect()


Running Test Routine
Running UR Test
Running Gripper Test
Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 
Position of dxl ID: 1 set to 549 
Position of dxl ID: 2 set to 490 
Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 


In [7]:
#Kinematic Model of the Robot in the Robotics Toolbox for Python (RTB) 
# Not needed but can be helpful for testing
#import RTB_Model as r
    

In [8]:
# unable to run last line, ''UR5' object has no attribute 'ikine_LMS' -Andrea
import RTB_Model as r
r = r.RTB_Model() #-- commneted out since I changed the class to an import -Jensen
r.getCameraFrame()
r.initSwiftEnv()
r.addSwiftBox([0,0,0.5])
r.simulateSwiftRobot()

[[1.   0.   0.   0.  ]
 [0.   1.   0.   0.25]
 [0.   0.   1.   0.25]
 [0.   0.   0.   1.  ]]
IKSolution: q=[1.461, -1.408, 2.529, 2.021, -3.032, 0], success=True, iterations=13, searches=1, residual=5.1e-11
[ 0.9250245  -1.95476876  2.51327412 -0.47996554  0.95993109  2.99673033]
[ 0.92506869 -1.9547237   2.5132754  -0.47975943  0.95960208  2.99648334]
[ 0.92536714 -1.95441938  2.51328404 -0.47836744  0.9573801   2.99481531]
[ 0.92614477 -1.95362645  2.51330654 -0.47474045  0.9515905   2.99046907]
[ 0.92759577 -1.95214689  2.51334852 -0.46797279  0.94078759  2.98235936]
[ 0.92988493 -1.94981269  2.51341476 -0.45729583  0.92374443  2.9695651 ]
[ 0.93314903 -1.94648435  2.51350921 -0.44207159  0.89944267  2.95132184]
[ 0.9374982  -1.94204959  2.51363505 -0.42178643  0.86706238  2.92701404]
[ 0.94301729 -1.93642188  2.51379475 -0.3960446   0.82597185  2.8961675 ]
[ 0.94976723 -1.92953911  2.51399006 -0.36456192  0.77571745  2.85844166]
[ 0.9577864  -1.92136212  2.5142221  -0.32715936  0.7

In [10]:
#Needed for camera use!! -Jensen
import RealSense as real

In [11]:
pipe = rs.pipeline()
config = rs.config()
profile = pipe.start()
try:
  for i in range(0, 100):
    frames = pipe.wait_for_frames()
    for f in frames:
      print(f.profile)
finally:
    pipe.stop()

<pyrealsense2.[video_]stream_profile: Depth(0) 848x480 @ 10fps Z16>
<pyrealsense2.[video_]stream_profile: Color(0) 848x480 @ 10fps RGB8>
<pyrealsense2.[video_]stream_profile: Depth(0) 848x480 @ 10fps Z16>
<pyrealsense2.[video_]stream_profile: Color(0) 848x480 @ 10fps RGB8>
<pyrealsense2.[video_]stream_profile: Depth(0) 848x480 @ 10fps Z16>
<pyrealsense2.[video_]stream_profile: Color(0) 848x480 @ 10fps RGB8>
<pyrealsense2.[video_]stream_profile: Depth(0) 848x480 @ 10fps Z16>
<pyrealsense2.[video_]stream_profile: Color(0) 848x480 @ 10fps RGB8>
<pyrealsense2.[video_]stream_profile: Depth(0) 848x480 @ 10fps Z16>
<pyrealsense2.[video_]stream_profile: Color(0) 848x480 @ 10fps RGB8>
<pyrealsense2.[video_]stream_profile: Depth(0) 848x480 @ 10fps Z16>
<pyrealsense2.[video_]stream_profile: Color(0) 848x480 @ 10fps RGB8>
<pyrealsense2.[video_]stream_profile: Depth(0) 848x480 @ 10fps Z16>
<pyrealsense2.[video_]stream_profile: Color(0) 848x480 @ 10fps RGB8>
<pyrealsense2.[video_]stream_profile: Dep

In [12]:

# robot_model = RTB_Model()
real = real.RealSense() #-- not need since Imported --Jensen
real.initConnection()
# real.getPCD(True)
# real.extrinsics = np.array(robot_model.getCameraFrame())
# print(real.extrinsics)
pcd,rgbdImage = real.getPCD(False)
depthImage,colorImage = rgbdImage.depth,rgbdImage.color
real.displayImages(colorImage,depthImage)
# real.displayPCD(pcd)
real.pipe.stop()

# o3d.visualization.draw()
# r.displayStream()
# finally:
#     r.pipe.stop()
# profile = pipe.start(config)
# depth_sensor = profile.get_device().first_depth_sensor()
# print(
# r.displayStream(pipe,config)


NameError: name 'np' is not defined

In [ ]:
# real = RealSense()
# real.initConnection()
# real.displayStream()

In [ ]:

robot_model = r.RTB_Model()
real = real.RealSense()
real.extrinsics = np.array(robot_model.getCameraFrame())
print(real.extrinsics)
pcd = o3d.io.read_point_cloud("pcd1.pcd")
real.displayPCD(pcd)


In [ ]:
#Needed for the experiment --Jensen
import Block 

In [ ]:
#Needed for the experiment I think --Jensen
import ObjectDetection

In [ ]:
#Needed 
import MotionPlanner

In [ ]:
# TASK PLANNER CODE

import blocksDomain
import blocksProblem
#contiuation of code above --Jesnen
import TaskPlanner

In [ ]:
# EXPERIMENT STEP (0) - Hardware Discovery
# Call this once to intialize serial connections to ur and gripper

robotIP = "192.168.0.6"
con = rtde_control.RTDEControlInterface(robotIP)
rec = rtde_receive.RTDEReceiveInterface(robotIP)

In [ ]:
# EXPERIMENT STEP (0) - Gripper Discovery
# To list serial ports of the motor interface
# $ python -m serial.tools.list_ports
servoPort = "/dev/ttyACM0"
gripperController = Motors(servoPort)
gripperController.torquelimit(600) # used to be 600
gripperController.speedlimit(100)
ur = UR5_Interface()
ur.gripperController = gripperController

In [ ]:
# EXPERIMENT STEP (1) - Hardware Interface Initialization
try:
    # robotIP = "192.168.0.6"
    # con = rtde_control.RTDEControlInterface(robotIP)
    # rec = rtde_receive.RTDEReceiveInterface(robotIP)
    # ur = UR5_Interface()
    ur.c = con
    ur.r = rec
    ur.gripperController = gripperController
except Exception as e:
    raise(e)
else:
    print("UR5 + Gripper Interface Established")

In [ ]:
ur.c.disconnect()
ur.r.disconnect()

In [ ]:
# EXPERIMENT STEP (2) - Hardware Test - Raises gripper 1 cm and open's closes gripper
ur.testRoutine()

In [ ]:
# EXPERIMENT STEP (3) - Initialize connection to RealSense
real = RealSense()
real.initConnection()
# real.displayStream()

In [ ]:
# EXPERIMENT STEP (4) - RealSense and YOLO Initialization
# robot_model steps can deleted once extrinsic-free transforms are verfied to be correct
# robot_model = RTB_Model()
# robot_model.setJointAngles(ur.getJointAngles())
try:
    detector = ObjectDetection(real,None,moveRelative = True)
except Exception as e:
    detector.real.pipe.stop()
    raise(e)

In [ ]:
ur.openGripper()

In [ ]:
# EXPERIMENT STEP (5) - Image Measurements, Segmentation, and Processing into Blocks
urPose = ur.getPose()
pcd,rgbdImage = detector.real.getPCD()
depthImage,colorImage = rgbdImage.depth,rgbdImage.color
blocks = detector.getBlocksFromImages(colorImage,depthImage,urPose,display = True)
for block in blocks:
    print(f"{block.name}:")
    print(f"CamFrameCoords: {block.camFrameCoords}")
    print(f"GripperFrameCoords: {block.gripperFrameCoords}")
    print(f"WorldFrameCoords: {block.worldFrameCoords}")

In [ ]:
# EXPERIMENT STEP (4) - Displaying PCD
detector.displayWorld(pcd,blocks)

In [ ]:
ur.openGripper()

In [ ]:
# EXPERIMENT STEP (5) - Task Planning
planner = TaskPlanner(blocks)
goalDict = {"on":[("blueBlock","yellowBlock")]}
steps = planner.generatePlan(goalDict)
print(steps)
for block in blocks:
    print(f"{block.name} - {list(block.gripperFrameCoords)}")


In [ ]:
# EXPERIMENT STEP (6) - Grasping blocks at each position in steps, returning to start position, moving to release position, moving back to start, opening gripper
sleepRate = 0.75
def projectToWorldCoords(gripperFrameCoords):
    # given a goal position in gripper coords returns the displacements from the current pose in world coords
    xB,yB,zB = gripperFrameCoords
    # subtract 0.165 from block position in gripper frame to account for gripper length
    zB -= 0.155
    currentPose = ur.getPose() #SE3 Object
    # print(f"Current Pose:\n{currentPose*1000}")
    R = currentPose.R 
    pX,pY,pZ = tuple(currentPose.t)
    # xB,yB,zB here is the block position in the gripper frame which is aligned with the optoforce frame
    P_goal = np.matmul(R,np.array([xB,yB,zB]).T)  # relative position of the block in world coordinates
    print(f"P_goal:\n{P_goal}")
    dX,dY,dZ = tuple(P_goal) # quantities and directions the the gripper frame should be incremented to be centered at the block 
    return dX,dY,dZ
    
def moveToBlock(blockPos):
    # would be better if this was block object
    # :blockPos is coord in gripper frame
    dX,dY,dZ = projectToWorldCoords(blockPos) # goalPose in world coordinates
    homePose = ur.getPose()
    dZ  += 7/1000 # up 7 mm to avoid hitting lower block
    goal1 = copy.deepcopy(homePose)
    goal1.t[2] += dZ
    ur.moveL(goal1)
    time.sleep(sleepRate)
    goal2 = goal1
    goal2.t[0] += dX
    goal2.t[1] += dY
    ur.moveL(goal2)
    time.sleep(sleepRate)
    
def moveBackFromBlock(homePose):    
    currentPose = ur.getPose()
    # Move up 3 mm to avoid raise block to prevent friction from toppling lower block
    goal1 = copy.deepcopy(currentPose)
    goal1.t[2] += 3/1000
    ur.moveL(goal1)
    time.sleep(sleepRate)
    currentPose = ur.getPose()
    dX,dY,dZ = tuple(homePose.t - currentPose.t)
    # Move in the XY Plane then Z Axis
    goal2 = copy.deepcopy(currentPose)
    goal2.t[0] += dX
    goal2.t[1] += dY
    ur.moveL(goal2)
    time.sleep(sleepRate)
    # Move in Z Axis back to home
    goal3 = copy.deepcopy(goal2)
    goal3.t[2] += dZ
    ur.moveL(goal3)
    time.sleep(sleepRate)


    
goalBlock = blocks[1]
blockLength = 0.02
releaseCoords = goalBlock.gripperFrameCoords + goalBlock.getWorldFrameVerticalInGripper(blockLength)
verticalDist = 0.02
gX,gY,gZ = tuple(goalBlock.urPose.t)
res = np.matmul(goalBlock.urPose.R,(sm.SE3.Trans([gX,gY,gZ+verticalDist]).t - goalBlock.urPose.t))
# print(f"res: {projectToWorldCoords(res)} ")
# ur.openGripper() # Open gripper
# ur.testRoutine()
# homePose = ur.getPose()
'''
for step in steps:
    # Grasp and Move Home Step
    grabPos,releasePos = step
    moveToBlock(grabPos) 
    print("Done moving to block")
    ur.closeGripper(9) 
    time.sleep(sleepRate)
    moveBackFromBlock(homePose)
    moveToBlock(releasePos)
    ur.closeGripper(55)
    moveBackFromBlock(homePose)
    ur.openGripper()
'''

In [ ]:
ur.c.disconnect()
ur.r.disconnect()
time.sleep(2)
try:
    robotIP = "192.168.0.6"
    con = rtde_control.RTDEControlInterface(robotIP)
    rec = rtde_receive.RTDEReceiveInterface(robotIP)
    #ur = UR5_Interface() -- commented out since when imported it is defined as ur. if you run this before importing uncommnet --Jensen
    ur.c = con
    ur.r = rec
    ur.gripperController = gripperController
    time.sleep(5)
    ur.testRoutine()
except Exception as e:
    raise(e)
else:
    print("UR5 + Gripper Interface Established")



In [ ]:
planner.locPositions["loc-a"]

In [ ]:
ur.moveL(homePose)

In [ ]:
ur.openGripper()

In [ ]:
ur.moveL(homePose)

In [ ]:
ur.openGripper()

In [ ]:
# MAY WANT TO BREAK THIS UP INTO SEPARATE CELLS
import UR5_Interface as ur 
def runRoutine():
    print("1")
    try:
        #ur = UR5_Interface()
        ur.c = con
        ur.r = rec
        ur.gripperController = gripperController
    except Exception as e:
        raise(e)
        
    print("2")
    # robot_model = RTB_Model()
    # robot_model.setJointAngles(ur.getJointAngles())
    # Set joint angles so object detection has correct extrinsics
    
    # Instantiates ObjectDetection object which intializes a connection to the realsense
    detector = ObjectDetection(robot_model,moveRelative = True)
    
    try:
        
        ur.openGripper()
        print(f"Nth Frame Pose:\n{np.array(ur.getPose())}")
        # Takes images for display
        pcd,rgbdImage = detector.real.getPCD()
        depthImage,colorImage = rgbdImage.depth,rgbdImage.color
        # detector.real.displayImages(depthImage,colorImage)
        
        blocks = detector.getBlocksFromImages(colorImage,depthImage)
        for block in blocks:
            print(f"{block.name}:")
            print(f"CamFrameCoords: {block.camFrameCoords}")
            print(f"GripperFrameCoords: {block.gripperFrameCoords}")
            print(f"WorldFrameCoords: {block.worldFrameCoords}")
        m = MotionPlanner(blocks,moveRelative = True)
        # Interface to the UR5
        m.ur = ur 
        # m.rtb_model = robot_model
        redBlock,yellowBlock,blueBlock = blocks
        redPCD,yellowPCD,bluePCD = redBlock.blockPCD,yellowBlock.blockPCD,blueBlock.blockPCD
        redAABB,yellowAABB,blueAABB = redBlock.blockAABB,yellowBlock.blockAABB,blueBlock.blockAABB,
        detector.real.displayPCD([redPCD,yellowPCD,bluePCD,redAABB,yellowAABB,blueAABB])
        m.runMovement()
        # time.sleep(3)
        # ur.closeGripper(10)
        
    except Exception as e:
        raise(e)
    finally:
        detector.real.pipe.stop()
        # ur.c.disconnect()
        # ur.r.disconnect()
runRoutine()

In [ ]:
ur.c.disconnect()
ur.r.disconnect()

robot_model = RTB_Model()
robot_model.plotRobot()
x,y,z = [],[],[]
for point in redPCD.points:
    x.append(point[0])
    y.append(point[1])
    z.append(point[2])

print(np.mean(x))
print(np.mean(y))
print(np.mean(z))
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(x,y,z)
plt.show()

In [ ]:
(real.extrinsics is None) == False